# Personalised PageRank-GCN(PGCN) 
For:

Learning from the Dark: Boosting Graph Convolutional Neural Networks wit Diverse Negative Samples.

This code can be run directly on colab

Getting negative samples **based on Personalised PageRank** refer to this article:

[Understanding Negative Sampling in Graph Representation Learning](https://arxiv.org/pdf/2005.09863.pdf)

# 0 Preparing

In [1]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-geometric

import torch
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

     |████████████████████████████████| 3.0 MB 2.4 MB/s 
     |████████████████████████████████| 1.6 MB 2.3 MB/s 
     |████████████████████████████████| 222 kB 4.0 MB/s 
     |████████████████████████████████| 376 kB 39.3 MB/s 
     |████████████████████████████████| 45 kB 2.2 MB/s 


## 0.1 Functions

###0.1.1  PageRank

In [2]:
  import tensorflow as tf
  def pagerank(graph,adjdense, damp_rate = 0.2):

    # node id must from 0 to any nature number.
    node_ids = sorted([id for id in graph.nodes]);
    assert node_ids == list(range(len(node_ids)));
    weights = tf.constant(adjdense);
    # normalize adjacent matrix line by line.
    line_sum = tf.math.reduce_sum(weights, axis = 1, keepdims = True) + 1e-6;
    normalized = weights / line_sum;
    # dampping vector.
    dampping = tf.ones((len(graph.nodes),), dtype = tf.float32);
    dampping = dampping / tf.constant(len(graph.nodes), dtype = tf.float32);
    dampping = tf.expand_dims(dampping, 0); # line vector.
    # learning pagerank.
    v = dampping;
    while True:
      v_updated = (1 - damp_rate) * tf.linalg.matmul(v, normalized) + damp_rate * dampping;
      d = tf.norm(v_updated - v);
      if tf.equal(tf.less(d,1e-6),True): break;
      v = v_updated;
    # edge weight is pagerank.
    weights = weights * tf.tile(v,(tf.shape(weights)[0],1));
    line_sum = tf.math.reduce_sum(weights, axis = 1, keepdims = True) + 1e-6;
    normalized = weights / line_sum;
    return normalized.numpy()

### 0.1.2 Basic GCN MessagePassing Layer

In [3]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
from torch_scatter import gather_csr, scatter
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add')  # "Add" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3-5: Start propagating messages.
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j, edge_index, size):
        # x_j has shape [E, out_channels]
        # edge_index has shape [2, E]

        # Step 3: Normalize node features.
        row, col = edge_index
        deg = degree(row, size[0], dtype=x_j.dtype)  # [N, ]
        deg_inv_sqrt = deg.pow(-0.5)   # [N, ]
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        return norm.view(-1, 1) * x_j
            
    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels]

        # Step 5: Return new node embeddings.
        return aggr_out

# 1. Load datasets

In [4]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.datasets import KarateClub

# dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())
dataset = Planetoid(root='data/Planetoid', name='Citeseer', transform=NormalizeFeatures())
# dataset = Planetoid(root='data/Planetoid', name='Pubmed')

#dataset = KarateClub()
print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Processing...
Done!

Dataset: Citeseer():
Number of graphs: 1
Number of features: 3703
Number of classes: 6

Data(edge_index=[2, 9104], test_mask=[3327], train_mask=[3327], val_mask=[3327], x=[3327, 3703], y=[3327])
Number of nodes: 3327
Number of edges: 9104
Average node degree: 2.74
Number of training nodes: 120
Training node label rate: 0.04
Contains isolated nodes: True
Contains self-loops: False
Is undirected: True


## 1.1 Calculating the maximum connected subgraph

In [5]:
from torch_geometric.utils import to_networkx, from_networkx
import networkx as nx

# transform data from torch_geometric to networkx
G = to_networkx(data, to_undirected=True)
# Get the index of all points of the max-connected subgraph
Subnode = max(nx.connected_components(G), key=len)
print(len(Subnode))
# Get the subgraph
SubGraph = G.subgraph(Subnode)
SubnodeList = list(Subnode)
print(SubnodeList)

2120
[1, 5, 8, 10, 12, 13, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 38, 39, 40, 42, 43, 44, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 61, 62, 65, 66, 69, 70, 72, 75, 76, 77, 78, 79, 80, 81, 83, 84, 87, 88, 90, 91, 92, 93, 95, 96, 98, 99, 100, 101, 103, 104, 105, 106, 107, 110, 113, 114, 115, 118, 119, 122, 123, 124, 126, 128, 130, 131, 132, 134, 135, 136, 137, 138, 142, 144, 147, 148, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 162, 167, 168, 169, 170, 172, 173, 177, 178, 180, 181, 184, 186, 188, 189, 190, 191, 194, 195, 197, 198, 200, 201, 203, 204, 205, 206, 208, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 224, 226, 227, 228, 229, 230, 231, 232, 234, 236, 237, 240, 241, 242, 243, 244, 246, 247, 249, 250, 252, 253, 254, 255, 258, 259, 260, 263, 265, 266, 267, 268, 269, 272, 273, 274, 280, 285, 286, 287, 289, 292, 293, 294, 298, 300, 302, 303, 304, 307, 308, 311, 312, 313, 314, 316, 317, 318, 319, 321, 322, 325, 330, 331, 332, 333,

## 1.2 Transform data from networkx to torch_geometric

In [6]:
PyTSubGraph = from_networkx(SubGraph)
SubGraph = to_networkx(PyTSubGraph, to_undirected=True)

PyTSubGraph.test_mask = data.test_mask[SubnodeList]
PyTSubGraph.train_mask = data.train_mask[SubnodeList]
PyTSubGraph.val_mask = data.val_mask[SubnodeList]
PyTSubGraph.x = data.x[SubnodeList]
PyTSubGraph.y = data.y[SubnodeList]

# Get number of nodes
num_nodes = PyTSubGraph.num_nodes
# Get edge_index
edge_index = PyTSubGraph.edge_index
edge_value = torch.ones(edge_index.size(1), device=edge_index.device)
# Transforming the adjacency matrix from a sparse transpose matrix to a dense matrix
adjdense = torch.sparse.FloatTensor(edge_index, edge_value, torch.Size([num_nodes,num_nodes])).to_dense()
print(adjdense.shape)

torch.Size([2120, 2120])


## 1.3 Generate candidates using Personalised PageRank

In [7]:
from collections import defaultdict
import random
import heapq
#Pagerank
pr=pagerank(SubGraph,adjdense.numpy(),0.2)

candidates = defaultdict(list)
SelNodeList = [] 
for i in range(PyTSubGraph.x.shape[0]):
    #GET index of non-adj nodes for a selected node
    nonadjIndex = torch.nonzero(adjdense[i]==0).numpy()
    nonadjIndex = list(nonadjIndex.reshape(-1))
    nonadj_pr = pr[i][nonadjIndex]
    if sum(nonadj_pr) == 0:
        selectlist = random.sample(nonadjIndex, 50)
    else: 
    # Get the index of the maximum 100 values
        max_number = heapq.nlargest(50, nonadj_pr) 
        selectlist = []
        for t in max_number:
            index = nonadj_pr.index(t)
            selectlist.append(index)
            nonadj_pr[index] = 0
    candidates[i].extend(selectlist)
print(candidates)

defaultdict(<class 'list'>, {0: [452, 1770, 930, 1388, 1088, 44, 1540, 225, 1765, 2000, 1516, 1236, 2082, 1152, 1659, 1879, 2013, 1135, 1253, 276, 481, 268, 648, 941, 1155, 1758, 1498, 354, 710, 1538, 1769, 1178, 296, 21, 2017, 2095, 474, 909, 606, 413, 386, 9, 157, 1980, 1578, 25, 1978, 1329, 1118, 1351], 1: [78, 396, 1849, 692, 514, 1699, 1684, 602, 1132, 380, 1225, 1012, 1029, 87, 315, 818, 1637, 1298, 1531, 1698, 666, 1323, 703, 1793, 1104, 1811, 522, 529, 1841, 997, 1056, 542, 5, 223, 1159, 1735, 1779, 205, 1454, 453, 956, 1287, 1289, 886, 833, 566, 1158, 219, 1421, 2046], 2: [485, 1578, 1850, 635, 651, 1783, 1605, 64, 1157, 1924, 451, 1835, 996, 466, 86, 1513, 102, 1733, 803, 791, 1327, 838, 1499, 776, 1378, 1298, 427, 541, 1726, 549, 1816, 1855, 1098, 1216, 1895, 1918, 1616, 2052, 1307, 1351, 1017, 1618, 1301, 144, 2094, 1565, 1199, 1664, 1480, 2055], 3: [1250, 717, 689, 1701, 1213, 1681, 493, 1606, 1590, 1464, 379, 890, 1248, 131, 1528, 1029, 772, 106, 239, 997, 1797, 170, 648,

# 2 Model Definition
## 2.1 Defining the MCGCN model

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import time


class PageRankGCN(torch.nn.Module):
    def __init__(self, hidden_channels,num_layers,num_nodes,candidates,adjdense):
        super(PageRankGCN, self).__init__()
        # torch.manual_seed(12345)

        self.num_layers = num_layers
        self.CONVs = torch.nn.ModuleList()
        self.RCONVs = torch.nn.ModuleList()
        self.NegRates = nn.ParameterList()
        for layer in range(self.num_layers-1):
            if layer == 0:
                self.CONVs.append(GCNConv(dataset.num_features, hidden_channels))
                self.RCONVs.append(GCNConv(dataset.num_features, hidden_channels))
                self.NegRates.append(nn.Parameter(torch.FloatTensor([1])))
            else:
                self.CONVs.append(GCNConv(hidden_channels, hidden_channels))
                self.RCONVs.append(GCNConv(hidden_channels, hidden_channels))
                self.NegRates.append(nn.Parameter(torch.FloatTensor([1])))
        self.CONVs.append(GCNConv(hidden_channels, dataset.num_classes))
        self.RCONVs.append(GCNConv(hidden_channels, dataset.num_classes))
        self.NegRates.append(nn.Parameter(torch.FloatTensor([1])))

        self.NegRate = 1
        self.candidates = candidates
        self.adjdense = adjdense
        self.num_nodes = num_nodes

    def forward(self, x, edge_index,Pagedge_index_list,train_neg_rate):
        for layer in range(self.num_layers):
          posi_x = self.CONVs[layer](x, edge_index)
          if len(Pagedge_index_list) < self.num_layers:
            PageMatrix = self.GetPageMatrix()
            #change dense back to sparse
            Pagedge_index = torch.nonzero(PageMatrix).t()
            Pagedge_index_list.append(Pagedge_index)
          else:
            Pagedge_index = Pagedge_index_list[layer]

          nega_x = self.RCONVs[layer](x, Pagedge_index)
          if train_neg_rate: #NegRate is a trainable parameter
            x = posi_x - self.NegRates[layer] * nega_x
          else:
            x = posi_x - self.NegRate * nega_x

          if layer < (self.num_layers-1):
            x = x.relu()
            x = F.dropout(x, p=0.5, training=self.training)
        return x,Pagedge_index_list
       
    
    def GetPageMatrix(self):
        PageMatrix = torch.zeros((self.num_nodes,self.num_nodes))
        for i in range(self.num_nodes):
          AdjList = []
          AjdIndex = torch.nonzero(self.adjdense[i])
          AjdIndex = torch.reshape(AjdIndex, (-1,))
          AjdIndex = AjdIndex.tolist()
          AdjList.extend(AjdIndex)
          sampleAmount = len(AdjList)
          Indexlist = self.candidates[i]
          if sampleAmount > len(Indexlist):
              sampleAmount = len(Indexlist)
          RanIndex = random.sample(Indexlist, sampleAmount)
          PageMatrix[i][RanIndex] = 1
        return PageMatrix
        

## 2.2 Defining the MAD

In [9]:
def MAD(x):
  x_norm = x / x.norm(dim=1)[:, None]
  dist = 1-torch.mm(x_norm, x_norm.transpose(0,1))
  one1 = torch.ones_like(dist)
  zreo1 = torch.zeros_like(dist)
  dist_1 = torch.where(dist > 0, one1, zreo1)
  D = dist.sum(0)/dist_1.sum(0)
  one = torch.ones_like(D)
  zero = torch.zeros_like(D)
  D_1 = torch.where(D > 0, one, zero)
  Mad = D.sum()/D_1.sum()
  return Mad.detach().numpy()


#3 Training

In [11]:
import sys

def test(Pagedge_index_list,train_neg_rate):
      model.eval()
      out,_ = model(PyTSubGraph.x, edge_index,Pagedge_index_list,train_neg_rate)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[PyTSubGraph.test_mask] == PyTSubGraph.y[PyTSubGraph.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(PyTSubGraph.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc

#----------------
# Set training parameters
num_layers = 5
Runs = 11
Epochs = 201
#-----------------
#Set whether NegRate is a trainable parameter
#if False:
#   NegRate = 1
train_neg_rate = False
#-----------------

for num in range(1,Runs):
  model = PageRankGCN(hidden_channels=16,
          num_layers=num_layers,
          num_nodes=num_nodes,
          candidates=candidates,
          adjdense=adjdense)
  if num == 1 :
    print(model)
  print("num_layers",num_layers)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
  criterion = torch.nn.CrossEntropyLoss()
  for epoch in range(1, Epochs):
      start = time.time()
      Pagedge_index_list = []
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out,Pagedge_index_list = model(PyTSubGraph.x, edge_index,Pagedge_index_list,train_neg_rate)  
      loss = criterion(out[PyTSubGraph.train_mask], PyTSubGraph.y[PyTSubGraph.train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()
      end = time.time()
      if epoch == 1:
          print("---------------")
          print("Pagedge_index_shape",Pagedge_index_list[0].shape)
          print("Epoch time:",end-start)
          print("---------------")
      if epoch % 10 == 0:
          test_acc = test(Pagedge_index_list,train_neg_rate)
          print(f'num: {num:03d}, Epoch: {epoch:03d}, Loss: {loss:.4f},Test Accuracy: {test_acc:.4f},MAD: {MAD(out):.4f}')

PageRankGCN(
  (CONVs): ModuleList(
    (0): GCNConv(
      (lin): Linear(in_features=3703, out_features=16, bias=True)
    )
    (1): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (2): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (3): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (4): GCNConv(
      (lin): Linear(in_features=16, out_features=6, bias=True)
    )
  )
  (RCONVs): ModuleList(
    (0): GCNConv(
      (lin): Linear(in_features=3703, out_features=16, bias=True)
    )
    (1): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (2): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (3): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (4): GCNConv(
      (lin): Linear(in_features=16, out_features=6, bias=True)
    )
  )
  (NegRates): ParameterList(
      (0): Parameter 